In [9]:
import csv

import pandas as pd
import os

df = pd.read_csv(os.path.join('datasets', 'spam.csv'))
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [10]:
import numpy as np
npArr = df.to_numpy()
npArr

array([['ham',
        'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'],
       ['ham', 'Ok lar... Joking wif u oni...'],
       ['spam',
        "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"],
       ...,
       ['ham',
        'Pity, * was in mood for that. So...any other suggestions?'],
       ['ham',
        "The guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free"],
       ['ham', 'Rofl. Its true to its name']], dtype=object)

In [16]:
import nltk
import string
import re

stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.WordNetLemmatizer()

for i in range(len(npArr)):
    text = npArr[i][1]
    text = re.sub('[^\w\s@#$%^&*()_-]', '', text)
    text = text.lower()
    textList = text.split()
    textList = [lemmatizer.lemmatize(word) for word in textList if word not in set(stopwords)]
    text = ' '.join(textList)

    npArr[i][1] = text
npArr

array([['ham',
        'go jurong point crazy available bugis n great world la e buffet cine got amore wat'],
       ['ham', 'ok lar joking wif u oni'],
       ['spam',
        'free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry question(std txt rate)t&cs apply 08452810075over18s'],
       ...,
       ['ham', 'pity * mood soany suggestion'],
       ['ham',
        'guy bitching acted like id interested buying something else next week gave u free'],
       ['ham', 'rofl true name']], dtype=object)

In [17]:
# Separating Test and Train Data
TRAIN_PERCENTAGE = 0.7
TEST_PERCENTAGE = 1 - TRAIN_PERCENTAGE

hams = [[ham, textArray] for ham, textArray in npArr if ham == 'ham' ]
spams = [[spam, textArray] for spam, textArray in npArr if spam == 'spam']

# Get proportional values
percentage_hams = len(hams) / len(npArr) # Percentage of ham
percentage_spams = len(spams) / len(npArr)
percentage_spams, percentage_hams

(0.13406317300789664, 0.8659368269921034)

In [24]:
import random

test_spams = []
spam_amount = int((len(spams) * TEST_PERCENTAGE) // 1)
for i in range(spam_amount):
    test_index = random.randint(0, len(spams) - 1)
    test_spams.append(spams[test_index])
    spams.pop(test_index)

test_hams = []
ham_amount = int((len(hams) * TEST_PERCENTAGE) // 1)
for i in range(ham_amount):
    test_index = random.randint(0, len(hams) - 1)
    test_hams.append(hams[test_index])
    hams.pop(test_index)



(348, 54)

In [25]:
# Shuffle the datasets

testList = test_spams + test_hams
random.shuffle(testList)

trainList = spams + hams
random.shuffle(trainList)

testList, trainList

([['ham', 'yes think office lap room think thats last day didnt shut'],
  ['ham',
   'men always need beautiful intelligent caring loving adjustable cooperative wife law allows one wife'],
  ['ham',
   'hear loud scream &lt#&gt minute cause gyno shoving thing dont belong'],
  ['ham', 'call later dont network urgnt sm'],
  ['ham', 'u got person story'],
  ['ham', 'mean fat head'],
  ['ham', 'ok'],
  ['ham', 'thats cool want please'],
  ['ham',
   'u say leh course nothing happen lar say v romantic jus bit lor thk e nite scenery nice leh'],
  ['ham',
   'wow youre right didnt mean guess gave boston men changed search location nyc something changed cuz signin page still say boston'],
  ['ham', 'hey late im amk need drink tea coffee'],
  ['ham', 'designation software developer may get chennai)'],
  ['ham', 'ow u deyi paid 60400thousadi told u would call'],
  ['ham', 'well ball time make call'],
  ['ham',
   'cool breeze bright sun fresh flower twittering bird waiting wish u goodmorning &am

In [26]:
# Now, split into X and Y.
# X is EMAIL text
# Y is SPAM/HAM

trainX = [text for classification, text in trainList]
trainY = [classification for classification, text in trainList]

trainX

['ranjith cal drpd deeraj deepak 5min hold',
 'uworld site buying qbank buying self assessment also',
 'important customer service announcement premier',
 'sorry ill call later',
 'he apparently bffs carly quick',
 'join uk horniest dogging service u sex 2nite sign follow instruction txt entry 69888 nytec2a3lpmsg@150p',
 'remember',
 'xmas & new year eve ticket sale club day 10am till 8pm thurs fri & sat night week theyre selling fast',
 'im leaving house',
 'stupidits possible',
 'dizzamn aight ill ask suitemates get back',
 'sorry ill call later',
 'howz painhope u r fine',
 'unsubscribed service get ton sexy babe hunk straight phone go httpgotbabescouk subscription',
 'u make fb list',
 '8 latest g still scrounge ammo want give new ak try',
 'yeah whatever lol',
 'sir waiting call',
 'entered cabin pa said happy bday bos felt special askd 4 lunch lunch invited apartment went',
 'might accidant tookplace somewhere ghodbandar rd traffic move slovely plz slip &amp dont worry',
 'oh rea

In [29]:
# Same for testX and testY
testX = [text for classification, text in testList]
testY = [classification for classification, text in testList]

In [30]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer()

# Train vectors
X_train_tf = tf_idf.fit_transform(trainX)
X_train_tf = tf_idf.transform(trainX)

# Test vectors
X_test_tf = tf_idf.transform(testX)

In [31]:
# Naive Bayes model
from sklearn.naive_bayes import MultinomialNB

nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tf, trainY)

y_pred = nb_classifier.predict(X_test_tf)

In [34]:
import sklearn
print(sklearn.metrics.classification_report(testY, y_pred, target_names=['spam', 'ham']))
# Precision: (True Positives) / (True Positives + False Positives)
# Recall: (T

# Model name: tf_idf

              precision    recall  f1-score   support

        spam       0.92      1.00      0.96       348
         ham       1.00      0.44      0.62        54

    accuracy                           0.93       402
   macro avg       0.96      0.72      0.79       402
weighted avg       0.93      0.93      0.91       402



In [43]:
# Write datasets to a file
import csv
header = ['Category', 'Message']
with open(os.path.join('datasets', 'train.csv'), 'w', encoding='utf-8') as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(header)
    writer.writerows(trainList,)

with open(os.path.join('datasets', 'test.csv'), 'w', encoding='utf-8') as f:
    writer = csv.writer(f, lineterminator='\n')
    writer.writerow(header)
    writer.writerows(testList)

print("Saved files")

Saved files


In [44]:
# Save the model
import joblib
joblib.dump(tf_idf, 'tf_idf.pkl')

['tf_idf.pkl']